In [7]:
from __future__ import print_function 
import numpy as np 
import pandas as pd 


class TreeNode(object):
    def __init__(self, ids=None, children=[], entropy=0, depth=0):
        self.ids = ids           # chỉ số của dữ liệu trong nút này
        self.entropy = entropy   # độ bất định (entropy), sẽ được tính sau
        self.depth = depth       # độ sâu từ nút gốc
        self.split_attribute = None  # thuộc tính nào được chọn để chia, nếu không phải là lá
        self.children = children  # danh sách các nút con của nó
        self.order = None         # thứ tự của các giá trị của thuộc tính chia trong các nút con
        self.label = None         # nhãn của nút nếu nó là một nút lá

    def set_properties(self, split_attribute, order):
        self.split_attribute = split_attribute  # thiết lập thuộc tính chia
        self.order = order                      # thiết lập thứ tự

    def set_label(self, label):
        self.label = label  # thiết lập nhãn cho nút


def entropy(freq):
    # loại bỏ xác suất bằng 0
    freq_0 = freq[np.array(freq).nonzero()[0]]  # chỉ giữ lại các giá trị không bằng 0
    prob_0 = freq_0 / float(freq_0.sum())  # tính xác suất
    return -np.sum(prob_0 * np.log(prob_0))  # tính entropy


class DecisionTreeID3(object):
    def __init__(self, max_depth=10, min_samples_split=2, min_gain=1e-4):
        self.root = None  # nút gốc của cây
        self.max_depth = max_depth  # độ sâu tối đa của cây
        self.min_samples_split = min_samples_split  # số mẫu tối thiểu để chia
        self.Ntrain = 0  # số lượng mẫu trong tập huấn luyện
        self.min_gain = min_gain  # mức tăng thông tin tối thiểu

    def fit(self, data, target):
        self.Ntrain = data.count()[0]  # số lượng mẫu trong tập dữ liệu
        self.data = data  # lưu dữ liệu
        self.attributes = list(data)  # lưu danh sách thuộc tính
        self.target = target  # lưu nhãn mục tiêu
        self.labels = target.unique()  # các nhãn duy nhất

        ids = range(self.Ntrain)  # danh sách chỉ số của các mẫu
        self.root = TreeNode(ids=ids, entropy=self._entropy(ids), depth=0)  # tạo nút gốc
        queue = [self.root]  # hàng đợi để duyệt qua các nút

        while queue:
            node = queue.pop()  # lấy nút cuối cùng ra khỏi hàng đợi
            if node.depth < self.max_depth or node.entropy < self.min_gain:
                node.children = self._split(node)  # chia nút
                if not node.children:  # nếu là nút lá
                    self._set_label(node)  # thiết lập nhãn cho nút lá
                queue += node.children  # thêm các nút con vào hàng đợi
            else:
                self._set_label(node)  # thiết lập nhãn nếu đã đạt độ sâu tối đa

    def _entropy(self, ids):
        # tính entropy của một nút với chỉ số ids
        if len(ids) == 0: return 0
        ids = [i + 1 for i in ids]  # chỉ số trong pandas bắt đầu từ 1
        freq = np.array(self.target[ids].value_counts())  # đếm tần suất của các nhãn
        return entropy(freq)  # trả về entropy

    def _set_label(self, node):
        # tìm nhãn cho một nút nếu nó là nút lá
        target_ids = [i + 1 for i in node.ids]  # chỉ số nhãn mục tiêu
        node.set_label(self.target[target_ids].mode()[0])  # chọn nhãn phổ biến nhất

    def _split(self, node):
        ids = node.ids  # chỉ số của các mẫu trong nút
        best_gain = 0  # lưu thông tin tốt nhất
        best_splits = []  # lưu các phân chia tốt nhất
        best_attribute = None  # thuộc tính tốt nhất để chia
        order = None  # thứ tự của giá trị thuộc tính

        sub_data = self.data.iloc[ids, :]  # dữ liệu con cho các chỉ số hiện tại
        for i, att in enumerate(self.attributes):
            values = self.data.iloc[ids, i].unique().tolist()  # các giá trị duy nhất của thuộc tính
            if len(values) == 1: continue  # nếu chỉ có một giá trị, không cần chia
            splits = []  # danh sách chứa các phân chia

            for val in values:
                sub_ids = sub_data.index[sub_data[att] == val].tolist()  # chỉ số của các mẫu theo giá trị
                splits.append([sub_id - 1 for sub_id in sub_ids])  # lưu lại chỉ số mẫu

            # không chia nếu nút có số điểm quá nhỏ
            if min(map(len, splits)) < self.min_samples_split: continue
            
            # tính thông tin tăng
            HxS = 0
            for split in splits:
                HxS += len(split) * self._entropy(split) / len(ids)  # thông tin tăng

            gain = node.entropy - HxS  # tính độ tăng thông tin
            if gain < self.min_gain: continue  # dừng nếu độ tăng nhỏ

            # nếu có độ tăng tốt hơn, lưu lại
            if gain > best_gain:
                best_gain = gain 
                best_splits = splits
                best_attribute = att
                order = values

        # thiết lập thuộc tính chia cho nút
        node.set_properties(best_attribute, order)
        # tạo các nút con với chỉ số của các phân chia tốt nhất
        child_nodes = [TreeNode(ids=split,
                                 entropy=self._entropy(split), depth=node.depth + 1) for split in best_splits]
        return child_nodes  # trả về các nút con

    def predict(self, new_data):
        """
        :param new_data: một dataframe mới, mỗi dòng là một điểm dữ liệu
        :return: nhãn dự đoán cho mỗi dòng
        """
        npoints = new_data.count()[0]  # số lượng điểm dữ liệu
        labels = [None] * npoints  # khởi tạo danh sách nhãn
        for n in range(npoints):
            x = new_data.iloc[n, :]  # một điểm dữ liệu
            # bắt đầu từ nút gốc và duyệt qua các nút nếu chưa gặp nút lá
            node = self.root
            while node.children: 
                node = node.children[node.order.index(x[node.split_attribute])]  # đi đến nút con tương ứng
            labels[n] = node.label  # lưu nhãn dự đoán
            
        return labels  # trả về danh sách nhãn dự đoán


if __name__ == "__main__":
    df = pd.read_csv('F:/weather.csv', index_col=0, parse_dates=True)  # đọc dữ liệu từ tệp CSV
    X = df.iloc[:, :-1]  # tách dữ liệu vào X (tất cả các cột ngoại trừ cột cuối)
    y = df.iloc[:, -1]   # tách nhãn vào y (cột cuối cùng)
    tree = DecisionTreeID3(max_depth=3, min_samples_split=2)  # khởi tạo cây quyết định
    tree.fit(X, y)  # huấn luyện cây với dữ liệu
    print(tree.predict(X))  # in ra các nhãn dự đoán cho tập dữ liệu


['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']


C:\Users\fptshop\AppData\Local\Temp\ipykernel_9040\2811287592.py:133: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('F:/weather.csv', index_col=0, parse_dates=True)  # đọc dữ liệu từ tệp CSV
C:\Users\fptshop\AppData\Local\Temp\ipykernel_9040\2811287592.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.Ntrain = data.count()[0]  # số lượng mẫu trong tập dữ liệu
C:\Users\fptshop\AppData\Local\Temp\ipykernel_9040\2811287592.py:119: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object':  # Nếu cột là kiểu chuỗi
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])  # Mã hóa cột
        label_encoders[column] = le  # Lưu lại label encoder để có thể sử dụng sau

sk_tree = DecisionTreeClassifier(max_depth=3, min_samples_split=2)
sk_tree.fit(X, y)  # huấn luyện mô hình với dữ liệu
sk_predictions = sk_tree.predict(X)  # Dự đoán nhãn trên dữ liệu X

print("Dự đoán từ cây quyết định scikit-learn:", sk_predictions)  # in ra dự đoán từ mô hình scikit-learn

Dự đoán từ cây quyết định scikit-learn: ['no' 'no' 'yes' 'no' 'yes' 'no' 'yes' 'no' 'yes' 'yes' 'no' 'yes' 'yes'
 'no']


C:\Users\fptshop\AppData\Local\Temp\ipykernel_9040\361489701.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column])  # Mã hóa cột
C:\Users\fptshop\AppData\Local\Temp\ipykernel_9040\361489701.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column])  # Mã hóa cột
C:\Users\fptshop\AppData\Local\Temp\ipykernel_9040\361489701.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_